In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import time
import os
import shutil
import torchvision.transforms as T
import torch
from PIL import Image
from time import time
from zipfile import ZipFile
import seaborn as sb
import glob
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import skimage.io as io

Progress bar  and timer functions to monitor the progress of model

In [ ]:
def progress_bar(perc_comp):
    '''
    Takes float or int between 0 and 100. 
    '''
    bar_width = 100
    bar = '#' * int(perc_comp)
    bar += '-' * (100-int(perc_comp)) + ' ' + str(int(perc_comp)) + '% complete'
    if perc_comp < 100:
        print (bar, end='\r')
    else:
        print (bar)
        
def timer_func(func):
    '''
    This function shows the execution time of the function object passed
    '''
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s')
        return result
    return wrap_func

Load cleaned Label .csv

In [ ]:
TRAIN_FOLDER = "./data/train_images/"

traindata = pd.read_csv("./data/final.csv")
traindata.loc[:,'image'] = TRAIN_FOLDER + traindata['image']

outputdir = './results/'

if not os.path.exists(outputdir+"/train_images/"):
    os.makedirs(outputdir+"/train_images/")

original = './data/final.csv'
target = './final.csv'
shutil.copyfile(original, target)

Image Training Database was very large, hence after selecting the images we wanted to use during the analysis of the labels in the .csv file, we only extracted the images required

In [ ]:
member_list = traindata['image'].tolist()
for i in member_list:
    with ZipFile('./train_images.zip', 'r') as zipObj:
        zipObj.extract(str(i[-18:]), path="./data/train_images/", pwd=None)

Load pretrained yolov5l model

In [4]:
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True) #Choose from yolov5n yolov5s yolov5m yolov5l yolov5x

Using cache found in C:\Users\Common/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-4-22 torch 1.9.0 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


Cropping function

In [5]:
def crop(model, im_file):
    results = model([im_file])

    results = results.pandas().xyxy[0]
    try:
        xmin, xmax, ymin, ymax = int(results.xmin.min()), int(results.xmax.max()), int(results.ymin.min()), int(results.ymax.max())
        return Image.fromarray(cv2.cvtColor(cv2.imread(im_file), cv2.COLOR_BGR2RGB)[ymin:ymax][:,xmin:xmax]), results

    except ValueError:
        return Image.fromarray(cv2.cvtColor(cv2.imread(im_file), cv2.COLOR_BGR2RGB)), pd.DataFrame()   

In [8]:
t1 = time()
train_annotations = pd.DataFrame()
for i in range(len(traindata.image.values)):
    output = outputdir+"/train_images/"+traindata.species.values[i]+"/"+traindata.image.values[i].split('/')[-1]
    img, df = crop(yolo, traindata.image.values[i])
    img = T.functional.center_crop(img, (max(img.size),max(img.size)))
    img = T.functional.resize(img,size=(224,224))
    if len(df)>0:    
        train_annotations = train_annotations.append(pd.DataFrame(df.values,
                            columns=['xmin', 'ymin', 'xmax', 'ymax', 'conf', 'class', 'name'], 
                            index=[traindata.image.values[i]]*len(df)))
    else:    
        train_annotations = train_annotations.append(pd.DataFrame(df.values,
                            ))
    img.save(output)
    progress_bar(i/(len(traindata.image.values)-1)*100)   
t2 = time()
print(f'Executed in {(t2-t1):.4f}s')

train_annotations.to_csv('training_annotations.csv')